<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/xlm_roBERTa_large_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [ ]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 35.8 MB/s 
     |████████████████████████████████| 432 kB 67.4 MB/s 
     |████████████████████████████████| 7.6 MB 60.1 MB/s 
     |████████████████████████████████| 163 kB 69.7 MB/s 
     |████████████████████████████████| 212 kB 74.7 MB/s 
     |████████████████████████████████| 115 kB 60.1 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [ ]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
!awk NF < dump.txt > kant.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  3423k      0 --:--:-- --:--:-- --:--:-- 3417k


In [ ]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
!shuf -n1000 kant.txt > kant_teste.txt

In [ ]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-459e25212047eba2/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

exemplo de entrada

In [ ]:
tokenized_datasets["train"][4]

{'input_ids': [0, 3145, 11549, 9153, 8, 91234, 2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 32


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],    
)

In [ ]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15332
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9590


Step,Training Loss
500,2.803300
1000,2.601200
1500,2.449100
2000,2.387800
2500,2.250400
3000,2.182300
3500,2.138000
4000,2.095000
4500,2.021100
5000,1.980400




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 1h 12min 59s, sys: 46min 49s, total: 1h 59min 48s
Wall time: 1h 59min 36s


TrainOutput(global_step=9590, training_loss=2.038637462373322, metrics={'train_runtime': 7176.849, 'train_samples_per_second': 21.363, 'train_steps_per_second': 1.336, 'total_flos': 8937591649090560.0, 'train_loss': 2.038637462373322, 'epoch': 10.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1991
  Batch size = 8


Perplexity: 5.37


In [ ]:
eval_results

{'eval_loss': 1.6806443929672241,
 'eval_runtime': 25.6037,
 'eval_samples_per_second': 77.762,
 'eval_steps_per_second': 9.725,
 'epoch': 10.0}

In [ ]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [ ]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading configuration file ./KantaiBERT/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
 

In [ ]:
fill_mask("O rapaz olhou para o <mask> ")

[{'score': 0.2342052012681961,
  'token': 177122,
  'token_str': 'chão',
  'sequence': 'O rapaz olhou para o chão'},
 {'score': 0.09425853937864304,
  'token': 48682,
  'token_str': 'médico',
  'sequence': 'O rapaz olhou para o médico'},
 {'score': 0.07863438129425049,
  'token': 107047,
  'token_str': 'criado',
  'sequence': 'O rapaz olhou para o criado'},
 {'score': 0.05684969946742058,
  'token': 1108,
  'token_str': 'mar',
  'sequence': 'O rapaz olhou para o mar'},
 {'score': 0.05309111252427101,
  'token': 175194,
  'token_str': 'céu',
  'sequence': 'O rapaz olhou para o céu'}]

In [ ]:
fill_mask("A moça olhou para o <mask> ")

[{'score': 0.21469515562057495,
  'token': 48682,
  'token_str': 'médico',
  'sequence': 'A moça olhou para o médico'},
 {'score': 0.18374431133270264,
  'token': 177122,
  'token_str': 'chão',
  'sequence': 'A moça olhou para o chão'},
 {'score': 0.08436691015958786,
  'token': 118427,
  'token_str': 'marido',
  'sequence': 'A moça olhou para o marido'},
 {'score': 0.05522457882761955,
  'token': 107047,
  'token_str': 'criado',
  'sequence': 'A moça olhou para o criado'},
 {'score': 0.038226544857025146,
  'token': 123975,
  'token_str': 'corredor',
  'sequence': 'A moça olhou para o corredor'}]

In [ ]:
fill_mask("Comprou uma <mask> na loja. ")

[{'score': 0.27952688932418823,
  'token': 185350,
  'token_str': 'cadeira',
  'sequence': 'Comprou uma cadeira na loja.'},
 {'score': 0.19087837636470795,
  'token': 2349,
  'token_str': 'casa',
  'sequence': 'Comprou uma casa na loja.'},
 {'score': 0.059768617153167725,
  'token': 20533,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.02255195379257202,
  'token': 163468,
  'token_str': 'camisa',
  'sequence': 'Comprou uma camisa na loja.'},
 {'score': 0.019000867381691933,
  'token': 30092,
  'token_str': 'coisa',
  'sequence': 'Comprou uma coisa na loja.'}]

In [ ]:
fill_mask("A mulher não é <mask>. ")

[{'score': 0.14893464744091034,
  'token': 10964,
  'token_str': 'tudo',
  'sequence': 'A mulher não é tudo.'},
 {'score': 0.09813497215509415,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'A mulher não é homem.'},
 {'score': 0.06389394402503967,
  'token': 12479,
  'token_str': 'assim',
  'sequence': 'A mulher não é assim.'},
 {'score': 0.06253133714199066,
  'token': 52636,
  'token_str': 'mulher',
  'sequence': 'A mulher não é mulher.'},
 {'score': 0.057391174137592316,
  'token': 91276,
  'token_str': 'bonita',
  'sequence': 'A mulher não é bonita.'}]

In [ ]:
fill_mask("O homem não é <mask>. ")

[{'score': 0.11431986838579178,
  'token': 11155,
  'token_str': 'Deus',
  'sequence': 'O homem não é Deus.'},
 {'score': 0.1007039025425911,
  'token': 5799,
  'token_str': 'nada',
  'sequence': 'O homem não é nada.'},
 {'score': 0.07102750986814499,
  'token': 13343,
  'token_str': 'outro',
  'sequence': 'O homem não é outro.'},
 {'score': 0.06308335065841675,
  'token': 10964,
  'token_str': 'tudo',
  'sequence': 'O homem não é tudo.'},
 {'score': 0.06036978214979172,
  'token': 166899,
  'token_str': 'pecado',
  'sequence': 'O homem não é pecado.'}]

In [ ]:
fill_mask("Ele é um bom <mask>. ")

[{'score': 0.35848256945610046,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.2961038649082184,
  'token': 114789,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'},
 {'score': 0.09099055826663971,
  'token': 38260,
  'token_str': 'amigo',
  'sequence': 'Ele é um bom amigo.'},
 {'score': 0.06511688977479935,
  'token': 187397,
  'token_str': 'menino',
  'sequence': 'Ele é um bom menino.'},
 {'score': 0.047009408473968506,
  'token': 208823,
  'token_str': 'sujeito',
  'sequence': 'Ele é um bom sujeito.'}]

In [ ]:
fill_mask("Ela é uma boa <mask>. ")

[{'score': 0.20974639058113098,
  'token': 179879,
  'token_str': 'criatura',
  'sequence': 'Ela é uma boa criatura.'},
 {'score': 0.1937529593706131,
  'token': 55207,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.15737108886241913,
  'token': 51047,
  'token_str': 'dama',
  'sequence': 'Ela é uma boa dama.'},
 {'score': 0.12647342681884766,
  'token': 29956,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.10272571444511414,
  'token': 22068,
  'token_str': 'alma',
  'sequence': 'Ela é uma boa alma.'}]

In [ ]:
fill_mask("Faz de conta que ainda é <mask>. ")

[{'score': 0.35064294934272766,
  'token': 79426,
  'token_str': 'criança',
  'sequence': 'Faz de conta que ainda é criança.'},
 {'score': 0.11396527290344238,
  'token': 15234,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.06415959447622299,
  'token': 96872,
  'token_str': 'jovem',
  'sequence': 'Faz de conta que ainda é jovem.'},
 {'score': 0.043052539229393005,
  'token': 2167,
  'token_str': 'tempo',
  'sequence': 'Faz de conta que ainda é tempo.'},
 {'score': 0.028546230867505074,
  'token': 11721,
  'token_str': 'pouco',
  'sequence': 'Faz de conta que ainda é pouco.'}]

In [ ]:
fill_mask("Depois da tempestade vem a <mask>. ")

[{'score': 0.1844898909330368,
  'token': 27761,
  'token_str': 'lua',
  'sequence': 'Depois da tempestade vem a lua.'},
 {'score': 0.17028972506523132,
  'token': 88641,
  'token_str': 'calma',
  'sequence': 'Depois da tempestade vem a calma.'},
 {'score': 0.11623431742191315,
  'token': 158483,
  'token_str': 'esperança',
  'sequence': 'Depois da tempestade vem a esperança.'},
 {'score': 0.10408887267112732,
  'token': 26486,
  'token_str': 'paz',
  'sequence': 'Depois da tempestade vem a paz.'},
 {'score': 0.078693687915802,
  'token': 2436,
  'token_str': 'vida',
  'sequence': 'Depois da tempestade vem a vida.'}]

In [ ]:
fill_mask("Mais vale um pássaro na mão do que <mask> voando. ")

[{'score': 0.2897784113883972,
  'token': 286,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.2209114283323288,
  'token': 13343,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.13111107051372528,
  'token': 68378,
  'token_str': 'sair',
  'sequence': 'Mais vale um pássaro na mão do que sair voando.'},
 {'score': 0.07436677068471909,
  'token': 193,
  'token_str': 'ir',
  'sequence': 'Mais vale um pássaro na mão do que ir voando.'},
 {'score': 0.07350456714630127,
  'token': 139278,
  'token_str': 'morrer',
  'sequence': 'Mais vale um pássaro na mão do que morrer voando.'}]

In [ ]:
fill_mask("Não tinha <mask>, mas almoçou mesmo assim. ")

[{'score': 0.22069041430950165,
  'token': 204382,
  'token_str': 'almoço',
  'sequence': 'Não tinha almoço, mas almoçou mesmo assim.'},
 {'score': 0.09454771876335144,
  'token': 167569,
  'token_str': 'jantar',
  'sequence': 'Não tinha jantar, mas almoçou mesmo assim.'},
 {'score': 0.09057150781154633,
  'token': 26216,
  'token_str': 'café',
  'sequence': 'Não tinha café, mas almoçou mesmo assim.'},
 {'score': 0.04566102847456932,
  'token': 2167,
  'token_str': 'tempo',
  'sequence': 'Não tinha tempo, mas almoçou mesmo assim.'},
 {'score': 0.027481388300657272,
  'token': 124126,
  'token_str': 'companhia',
  'sequence': 'Não tinha companhia, mas almoçou mesmo assim.'}]

In [ ]:
fill_mask("Bebeu um copo d'água, pois tinha <mask>. ")

[{'score': 0.31207865476608276,
  'token': 19378,
  'token_str': 'sede',
  'sequence': "Bebeu um copo d'água, pois tinha sede."},
 {'score': 0.14831292629241943,
  'token': 149034,
  'token_str': 'frio',
  'sequence': "Bebeu um copo d'água, pois tinha frio."},
 {'score': 0.08418119698762894,
  'token': 30713,
  'token_str': 'calor',
  'sequence': "Bebeu um copo d'água, pois tinha calor."},
 {'score': 0.07569854706525803,
  'token': 18972,
  'token_str': 'dor',
  'sequence': "Bebeu um copo d'água, pois tinha dor."},
 {'score': 0.07360025495290756,
  'token': 1736,
  'token_str': 'sed',
  'sequence': "Bebeu um copo d'água, pois tinha sed."}]

In [ ]:
fill_mask("Sem saber, ele descobriu um <mask>. ")

[{'score': 0.5982560515403748,
  'token': 165801,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.128716379404068,
  'token': 137558,
  'token_str': 'secreto',
  'sequence': 'Sem saber, ele descobriu um secreto.'},
 {'score': 0.02407895028591156,
  'token': 83367,
  'token_str': 'perigo',
  'sequence': 'Sem saber, ele descobriu um perigo.'},
 {'score': 0.014363998547196388,
  'token': 166899,
  'token_str': 'pecado',
  'sequence': 'Sem saber, ele descobriu um pecado.'},
 {'score': 0.011051052249968052,
  'token': 11721,
  'token_str': 'pouco',
  'sequence': 'Sem saber, ele descobriu um pouco.'}]

In [ ]:
fill_mask("Pedro fez fortuna vendendo <mask>. ")

[{'score': 0.21244460344314575,
  'token': 10964,
  'token_str': 'tudo',
  'sequence': 'Pedro fez fortuna vendendo tudo.'},
 {'score': 0.13033029437065125,
  'token': 2349,
  'token_str': 'casa',
  'sequence': 'Pedro fez fortuna vendendo casa.'},
 {'score': 0.04641347378492355,
  'token': 36,
  'token_str': 'o',
  'sequence': 'Pedro fez fortuna vendendo o.'},
 {'score': 0.04113622009754181,
  'token': 10,
  'token_str': 'a',
  'sequence': 'Pedro fez fortuna vendendo a.'},
 {'score': 0.0357782319188118,
  'token': 51881,
  'token_str': 'dinheiro',
  'sequence': 'Pedro fez fortuna vendendo dinheiro.'}]

In [ ]:
fill_mask("Sem medo de ser <mask>. ")

[{'score': 0.13984815776348114,
  'token': 33587,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.11324557662010193,
  'token': 13323,
  'token_str': 'visto',
  'sequence': 'Sem medo de ser visto.'},
 {'score': 0.05894521623849869,
  'token': 29377,
  'token_str': 'preso',
  'sequence': 'Sem medo de ser preso.'},
 {'score': 0.051975131034851074,
  'token': 44053,
  'token_str': 'padre',
  'sequence': 'Sem medo de ser padre.'},
 {'score': 0.05156397074460983,
  'token': 178391,
  'token_str': 'ouvido',
  'sequence': 'Sem medo de ser ouvido.'}]